In [1]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
from scipy import optimize
import sys
sys.path.append('../SEIR_full/')
sys.path.append('..')
import SEIR_full as mdl
import SEIR_full.model_class as mdl
import datetime as dt

In [2]:
plt.style.use('dean_style.conf')

In [3]:
IL_pop = 9136000

In [4]:
with (open('../Data/parameters/indices.pickle', 'rb')) as openfile:
    ind = pickle.load(openfile)

# Imports model's predictions

In [5]:
with open('../Data/results/28_4_20_20_36.pickle','rb') as pickle_file:
    inter_data = pickle.load(pickle_file)

In [6]:
inter_data.keys()

dict_keys([('20@50_noschool_kid010', 'ub'), ('20@75_noschool_kid010', 'ub'), ('20@100_noschool_kid010', 'ub'), ('20@100', 'ub'), ('20@50_noschool_kid010', 'base'), ('20@75_noschool_kid010', 'base'), ('20@100_noschool_kid010', 'base'), ('20@100', 'base'), ('20@50_noschool_kid010', 'lb'), ('20@75_noschool_kid010', 'lb'), ('20@100_noschool_kid010', 'lb'), ('20@100', 'lb')])

In [8]:
region_age_ga_dict = mdl.get_opposite_dict(ind.GA,list(itertools.product(ind.G.values(),ind.A.values())))
region_index = {v:k for k,v in ind.G.items()}
init_region_pop = mdl.shrink_array_sum(mapping_dic=ind.region_dict,array_to_shrink=mdl.population_size)
init_region_age_pop = mdl.shrink_array_sum(mapping_dic=ind.region_age_dict,array_to_shrink=mdl.population_size)

In [9]:
init_region_age_pop.sum()

1.0

# Map dashboard

In [19]:
def map_table(res_mdl, inter_code, col_name='mean', start_day='2020-02-20'):
    
    tot_I = res_mdl['Is'] + res_mdl['Ia'] + res_mdl['Ie']
    date_range = pd.date_range(start_day, periods = tot_I.shape[0], freq='d')
    
    # Creating table for map, by region:
    table = pd.DataFrame(index = pd.MultiIndex.from_tuples(itertools.product(date_range, ind.G.values(),list(ind.A.values()) + ['total']),
                                                           names=['date','region','age']), 
                                     columns=[col_name])
    # filling df, cases per 1K
    for index in list(table.index):
#         print(index)
#         print(type(tot_I[int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]].sum()))
#         print([int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]])
        
        # if total area value needed
        if index[2] == 'total':
            table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),ind.region_dict[index[1]]].sum()
                                / init_region_pop[region_index[index[1]]]) * 1000
        else:
            table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),ind.region_age_dict[index[1],index[2]]].sum()
                                / init_region_age_pop[region_age_ga_dict[index[1], index[2]]]) * 1000
    
    # calculate weekly cases
    # 'daysoffset' will container the weekday, as integers
    table['daysoffset'] = table.apply(lambda x: x.name[0].weekday(),axis=1)
    # We apply, row by row (axis=1) a timedelta operation
    table['week_start'] = table.apply(lambda x: x.name[0] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    table.reset_index(inplace=True)
    table_groupped = table.groupby(['week_start','region','age'])[col_name]
    table_groupped = table_groupped.apply(lambda x: (x.max() + x.min())/2)
    table_groupped =  table_groupped.to_frame()
    table_groupped['scenario_idx'] = inter_code
    
    return(table_groupped[pd.to_datetime('today'):])

In [20]:
scens = [
    '20@50_noschool_kid010',
    '20@75_noschool_kid010',
    '20@100_noschool_kid010',
    '20@100',
]
model_sets = ['base']

In [21]:
inter_dict_map = {}

In [22]:
for scen in scens:
    for p in model_sets:
        inter_dict_map[scen] = map_table(inter_data[scen,p], 
                                         scen, col_name=p, start_day='2020-02-20').reset_index()
    

In [23]:
inter_df_list = [v for v in inter_dict_map.values()]
inter_df_concat = pd.concat(inter_df_list)

In [24]:
# Save
inter_df_concat.to_csv('../Data/results/map_data_upload_280420.csv')

# Age distribution

In [38]:
def age_distribution_table(res_mdl, inter_code, col_name='mean', start_day='2020-02-20'):
    
    tot_I = res_mdl['Is'] + res_mdl['Ia'] + res_mdl['Ie']
    date_range = pd.date_range(start_day, periods = tot_I.shape[0], freq='d')
    
    # Creating table for map, by region:
    table = pd.DataFrame(index = pd.MultiIndex.from_tuples(itertools.product(date_range,list(ind.A.values())),
                                                           names=['date','age']), 
                                     columns=[col_name])

    for index in list(table.index):
#         print(index)
#         print(type(tot_I[int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]].sum()))
#         print([int(np.where(date_range == index[0])[0]),mdl.region_dict[index[1]]])
        
        table.loc[index] = (tot_I[int(np.where(date_range == index[0])[0]),ind.age_dict[index[1]]].sum())*IL_pop
    
    # calculate weekly cases
    # 'daysoffset' will container the weekday, as integers
    table['daysoffset'] = table.apply(lambda x: x.name[0].weekday(),axis=1)
    # We apply, row by row (axis=1) a timedelta operation
    table['week_start'] = table.apply(lambda x: x.name[0] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    table.reset_index(inplace=True)
    table_groupped = table.groupby(['week_start','age'])[col_name]
    table_groupped = table_groupped.apply(lambda x: np.ceil((x.max() + x.min())/2))
    table_groupped =  table_groupped.to_frame()
    table_groupped['scenario_idx'] = inter_code
    
    return(table_groupped[pd.to_datetime('today'):])

In [39]:
def combine_ub_lb_ml(ub_df, lb_df, ml_df, col_list=['ub','lb','ml']):
    
    combined_df = ml_df.copy()
    combined_df = combined_df.merge(ub_df, left_index=True, right_index=True)
    combined_df = combined_df.merge(lb_df, left_index=True, right_index=True)
#     print(combined_df)
    # fixing LB/Mean/UB
    combined_df['new_lb'] = combined_df[col_list].apply(lambda x: x.min(), axis=1)
    combined_df['new_ub'] = combined_df[col_list].apply(lambda x: x.max(), axis=1)
    combined_df['new_ml'] = combined_df[col_list].apply(lambda x: sorted([x[0],x[1],x[2]])[1], axis=1)
    
    return combined_df[['new_lb','new_ub','new_ml','scenario_idx']]
    

In [40]:
scens = [
    '20@50_noschool_kid010',
    '20@75_noschool_kid010',
    '20@100_noschool_kid010',
    '20@100',
]
model_sets = ['base','ub','lb']

In [41]:
inter_dict_age_dis = {}

In [42]:
for scen in scens:
    curr_inter_dict = {}
    for p in model_sets:
        curr_inter_dict[p] = age_distribution_table(inter_data[scen,p], 
                                         scen, col_name=p, start_day='2020-02-20')


    # combining ub,lb,ml to one DF:

    inter_dict_age_dis[scen] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                    col_list=model_sets).reset_index()

In [43]:
inter_df_list_age_dis = [v for v in inter_dict_age_dis.values()]
inter_df_age_dis = pd.concat(inter_df_list_age_dis)

In [44]:
inter_df_age_dis

,week_start,age,new_lb,new_ub,new_ml,scenario_idx
0,2020-05-04,0-4,74.0,579.0,176.0,20@50_noschool_kid010
1,2020-05-04,10-19,228.0,1774.0,614.0,20@50_noschool_kid010
2,2020-05-04,20-29,95.0,951.0,287.0,20@50_noschool_kid010
3,2020-05-04,30-39,81.0,772.0,241.0,20@50_noschool_kid010
4,2020-05-04,40-49,41.0,775.0,124.0,20@50_noschool_kid010
...,...,...,...,...,...,...
112,2020-07-27,40-49,24323.0,139107.0,41320.0,20@100
113,2020-07-27,5-9,43827.0,102649.0,56850.0,20@100
114,2020-07-27,50-59,18119.0,96066.0,30977.0,20@100
115,2020-07-27,60-69,24963.0,97093.0,38998.0,20@100


In [45]:
# Save
inter_df_age_dis.to_csv('../Data/results/age_data_upload_280420.csv')

# Ventilators graph

In [46]:
def vents_graph(res_mdl, inter_code, tracking='Vents' ,col_name='mean', curr_vents=None, start_day='2020-02-20', 
                inter_start_date='2020-04-20', fix_vents=False):
    
    tot_vents = np.ceil(res_mdl[tracking].sum(axis=1) * IL_pop) 
    date_range = pd.date_range(start_day, periods = tot_vents.shape[0], freq='d')
    # fixing patients no.
    if fix_vents:
        model_vents_proj = tot_vents[int(np.where(date_range == inter_start_date)[0])]
        tot_vents += max(curr_vents - model_vents_proj, 0)

    
    # creates DF with results
    vents_dict = {}
    vents_dict['date'] = date_range
    vents_dict[col_name] = tot_vents
    vents_df = pd.DataFrame.from_dict(vents_dict)
    
    # 'daysoffset' will container the weekday, as integers
    vents_df['daysoffset'] = vents_df['date'].apply(lambda x: x.weekday())
    # We apply, row by row (axis=1) a timedelta operation
    vents_df['week_start'] = vents_df.apply(lambda x: x['date'] - dt.timedelta(days=x['daysoffset']), axis=1)
    
    vents_df_grop = vents_df.groupby('week_start')[col_name]
    vents_df_grop = vents_df_grop.max().to_frame()
    vents_df_grop['scenario_idx'] = inter_code
    return (vents_df_grop[pd.to_datetime('today'):])

In [47]:
scens = [
    '20@50_noschool_kid010',
    '20@75_noschool_kid010',
    '20@100_noschool_kid010',
    '20@100',
]
model_sets = ['base','ub','lb']

In [48]:
inter_dict_vents_dict = {}

In [49]:
for scen in scens:
    curr_inter_dict = {}
    for p in model_sets:
        curr_inter_dict[p] = vents_graph(inter_data[scen,p], 
                                         scen, col_name=p, curr_vents=113, start_day='2020-02-20', fix_vents=True)


    # combining ub,lb,ml to one DF:

    inter_dict_vents_dict[scen] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                    col_list=model_sets).reset_index()

In [50]:
inter_df_list_vents_dis = [v for v in inter_dict_vents_dict.values()]
inter_df_vents_dis = pd.concat(inter_df_list_vents_dis)

In [51]:
inter_df_vents_dis

,week_start,new_lb,new_ub,new_ml,scenario_idx
0,2020-05-04,105.0,201.0,108.0,20@50_noschool_kid010
1,2020-05-11,99.0,184.0,102.0,20@50_noschool_kid010
2,2020-05-18,93.0,179.0,100.0,20@50_noschool_kid010
3,2020-05-25,90.0,194.0,100.0,20@50_noschool_kid010
4,2020-06-01,93.0,221.0,101.0,20@50_noschool_kid010
5,2020-06-08,96.0,259.0,103.0,20@50_noschool_kid010
6,2020-06-15,102.0,310.0,106.0,20@50_noschool_kid010
7,2020-06-22,109.0,376.0,111.0,20@50_noschool_kid010
8,2020-06-29,118.0,460.0,119.0,20@50_noschool_kid010
9,2020-07-06,127.0,566.0,130.0,20@50_noschool_kid010


In [52]:
# Save
inter_df_vents_dis.to_csv('../Data/results/vents_data_upload_280420.csv')

# Hospitalization graph

In [53]:
scens = [
    '20@50_noschool_kid010',
    '20@75_noschool_kid010',
    '20@100_noschool_kid010',
    '20@100',
]
model_sets = ['base','ub','lb']

In [54]:
inter_dict_vents_dict = {}

In [55]:
for scen in scens:    
    curr_inter_dict = {}
    for p in model_sets:
        curr_inter_dict[p] = vents_graph(inter_data[scen,p], 
                                         scen, tracking='H', col_name=p, curr_vents=113, start_day='2020-02-20', fix_vents=True)


    # combining ub,lb,ml to one DF:

    inter_dict_vents_dict[scen] = combine_ub_lb_ml(curr_inter_dict['ub'],curr_inter_dict['lb'],curr_inter_dict['base'],
                                                    col_list=model_sets).reset_index()

In [56]:
inter_df_list_vents_dis = [v for v in inter_dict_vents_dict.values()]
inter_df_vents_dis = pd.concat(inter_df_list_vents_dis)

In [57]:
inter_df_vents_dis

,week_start,new_lb,new_ub,new_ml,scenario_idx
0,2020-05-04,110.0,996.0,339.0,20@50_noschool_kid010
1,2020-05-11,89.0,880.0,274.0,20@50_noschool_kid010
2,2020-05-18,79.0,921.0,240.0,20@50_noschool_kid010
3,2020-05-25,84.0,1075.0,241.0,20@50_noschool_kid010
4,2020-06-01,97.0,1312.0,264.0,20@50_noschool_kid010
5,2020-06-08,119.0,1638.0,302.0,20@50_noschool_kid010
6,2020-06-15,152.0,2067.0,352.0,20@50_noschool_kid010
7,2020-06-22,198.0,2618.0,418.0,20@50_noschool_kid010
8,2020-06-29,262.0,3314.0,500.0,20@50_noschool_kid010
9,2020-07-06,349.0,4181.0,601.0,20@50_noschool_kid010


In [58]:
# Save
inter_df_vents_dis.to_csv('../Data/results/hospitalization_data_upload_280420.csv')

# County id -> name mapper

In [268]:
region_name = {'11':'Jerusalem and sub.', '11_betshemesh':'Bet Shemesh', '21':'Zefat', '22':'Kinneret',
               '23':'Jezreel Valley', '24':'Akko', '29':'Ramat Hagolan', '31':'Haifa', '32':'Hadera',
               '41': 'Sharon', '42':'Petah Tiqwa', '43':'Ramla', '44':'Rehovot', '51_tlv':'Tel Aviv - Yafo',
               '51_bb': 'Bnei Brak', '51':'TLV suburbs', '61':'Ashqelon', '62':'Beer Sheva Jewish',
               '62_arab':'Beer Sheva Arabs','71': 'Judea and Samaria'}


In [269]:
region_name_df = pd.DataFrame.from_dict(region_name,orient='index')

In [271]:
region_name_df.to_csv('../Data/dashboard/region_name_mapper_county.csv')